In [1]:
@file:DependsOn("io.ktor:ktor-client-core:3.1.3")
@file:DependsOn("io.ktor:ktor-client-cio:3.1.3")
@file:DependsOn("io.ktor:ktor-client-content-negotiation:3.1.3")
@file:DependsOn("io.ktor:ktor-serialization-kotlinx-json:3.1.3")
@file:DependsOn("com.charleskorn.kaml:kaml-jvm:0.67.0")
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:import-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:account-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
%use dataframe

import ro.jf.funds.client.notebook.*

val client = FundsClient()
val user = client.ensureUserExists("Johann-18.2")
user

UserTO(id=f6b44c2b-6e79-4a2b-ad01-6f8546f7f04f, username=Johann-18.2)

In [2]:
import ro.jf.funds.account.sdk.AccountSdk
import ro.jf.funds.fund.sdk.FundSdk
import ro.jf.funds.importer.sdk.ImportSdk

val ACCOUNTS_YAML_FILE = "../../data/provision/accounts.yaml"
val FUNDS_YAML_FILE = "../../data/provision/funds.yaml"
val IMPORT_CONFIGURATION_JSON_FILE = "../../data/provision/import-configuration.json"
val IMPORT_CONFIGURATION_YAML_FILE = "../../data/provision/import-configuration.yaml"

val accountSdk = AccountSdk()
val fundSdk = FundSdk()
val importSdk = ImportSdk()


In [3]:
import ro.jf.funds.account.api.model.*
import ro.jf.funds.commons.model.*
import kotlinx.coroutines.runBlocking
import com.charleskorn.kaml.Yaml
import kotlinx.serialization.builtins.ListSerializer

val accounts = runBlocking {
    val existingAccounts = accountSdk.listAccounts(user.id).items
    val existingAccountNames = existingAccounts.map { it.name }.toSet()
    val newAccounts = File(ACCOUNTS_YAML_FILE).readText()
        .let { Yaml.default.decodeFromString(ListSerializer(CreateAccountTO.serializer()), it) }
        .filter { it.name !in existingAccountNames }
        .map { accountSdk.createAccount(user.id, it) }
    existingAccounts + newAccounts
}
accounts.map { it.toString() }.joinToString(separator = "\n")

AccountTO(id=7f8133b4-d181-4591-9e99-b4dcae69833b, name=Cash RON, unit=Currency(value=RON))
AccountTO(id=ad76ea45-14fc-42ea-bc4a-00f394e6c026, name=Cash EUR, unit=Currency(value=EUR))
AccountTO(id=90889731-60ee-4923-87ad-a29f67f338ea, name=ING RON, unit=Currency(value=RON))
AccountTO(id=5bf83963-5946-465b-ad47-be38204a9b4c, name=ING Economy, unit=Currency(value=RON))
AccountTO(id=51ab1cbb-079d-4a9c-bcf7-ec506ebecace, name=Food Coupons, unit=Currency(value=RON))
AccountTO(id=e268ce22-eabb-44f6-9441-f350985fe8e3, name=Revolut RON, unit=Currency(value=RON))
AccountTO(id=a7a1c167-a9fe-495b-8615-b1ba68497815, name=Other, unit=Currency(value=RON))
AccountTO(id=017b958f-30aa-4c90-90bb-0f9ea9c86060, name=BT RON, unit=Currency(value=RON))
AccountTO(id=6830b256-564a-4420-8c21-236a0c94f672, name=BT Economy RON, unit=Currency(value=RON))

In [4]:
import ro.jf.funds.fund.api.model.*
import kotlinx.coroutines.runBlocking

val funds = runBlocking {
    val existingFunds = fundSdk.listFunds(user.id).items
    val existingFundNames = existingFunds.map { it.name }.toSet()
    val newFunds = File(FUNDS_YAML_FILE).readText()
        .let { Yaml.default.decodeFromString(ListSerializer(CreateFundTO.serializer()), it) }
        .filter { it.name !in existingFundNames }
        .map { fundSdk.createFund(user.id, it) }
    existingFunds + newFunds
}
funds.map { it.toString() }.joinToString(separator = "\n")

FundTO(id=e5416a98-a6d0-4d04-9def-e3b8391cac22, name=Expenses)
FundTO(id=30f26512-e8bc-49ca-9c97-23ffb9d71037, name=Work Income)
FundTO(id=46cfa374-afa2-4d0d-9acf-8437a0132e3d, name=Savings)
FundTO(id=7656f916-fb80-4a9d-b10c-8ca6160950a5, name=Gifts)

In [5]:
import ro.jf.funds.importer.api.model.*
import kotlinx.serialization.json.Json

val dataConfigurationRawYaml: String = File(IMPORT_CONFIGURATION_YAML_FILE).readText()
val importConfiguration = Yaml.default.decodeFromString(ImportConfigurationTO.serializer(), dataConfigurationRawYaml)

In [6]:
import kotlinx.coroutines.delay
import kotlinx.coroutines.runBlocking
import kotlinx.datetime.Clock
import ro.jf.funds.importer.api.model.*
import java.io.File
import java.util.*
import kotlin.time.Duration

val csvFiles =
    listOf(File("../../data/wallet/2019"), File("../../data/wallet/2020"))
        .flatMap { it.listFiles()?.toList() ?: error("no files found") }
runBlocking {
    var importTask = importSdk.import(user.id, importConfiguration, csvFiles)
    val now: Instant = Clock.System.now()
    val timeout = 60.seconds
    while (importTask.status == ImportTaskTO.Status.IN_PROGRESS && Clock.System.now() - now < timeout) {
        delay(500)
        importTask = importSdk.getImportTask(user.id, importTask.taskId)
    }
    importTask
}


ImportTaskTO(taskId=73b96f95-0073-49eb-9f0e-1e4749554d54, status=COMPLETED, reason=null)